<a href="https://colab.research.google.com/github/Alecia113/NLP-Ex/blob/main/E10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise

## E1. Explain the difference between global and local attention with examples. 

**NOTE: Your E1 will NOT be marked if no example provided**

E1. 举例说明全局注意和局部注意之间的区别。 【例子】
注意：如果没有提供例子，你的E1将不会被评分。

Your Answer:

## E2. Scale Dot Product
Please change the **Dot Product** attention in the following code into **Scaled Dot Product** attention

**Dot Product:**

![Dot_Product](https://drive.google.com/uc?id=1QtBgCp53e_6A_vzaMFEo89GJbTxnXagJ)

**Scaled Dot Product:**

![Scaled_Dot_Product](https://drive.google.com/uc?id=1v6n9WChBVfy0mBG2yxK9MUvGKzVGCmOt)


比例点积
请将以下代码中的点积关注点改为比例点积关注点
【dot-product---> scaled dot product】
点积。
缩放点积。

In [2]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    
downloaded = drive.CreateFile({'id': '1AczfPH196NVsD8j9TwU-xN5XlBztEDy-'})
downloaded.GetContentFile('qna_chitchat_friendly.tsv')

import pandas as pd
df_friend = pd.read_csv('qna_chitchat_friendly.tsv', sep="\t")  #让来源==文件  导入
df_friend.sample(10)

,Question,Answer,Source,Metadata
666,Do you have any uncles?,I come from a long line of code.,qna_chitchat_Friendly,editorial:chitchat
73,You are boring as sin,Swing and a miss.,qna_chitchat_Friendly,editorial:chitchat
882,Can you show me how to play a game?,That's not one of my talents.,qna_chitchat_Friendly,editorial:chitchat
1595,What do you go by?,"Oh, I don't have a name.",qna_chitchat_Friendly,editorial:chitchat
6235,Well hi there,Hi!,qna_chitchat_Friendly,editorial:chitchat
8426,I want someone to love,I'm sending a hug your way.,qna_chitchat_Friendly,editorial:chitchat
1197,Where do you fall in terms of gender,That doesn't really apply to me.,qna_chitchat_Friendly,editorial:chitchat
9186,I wish I were dead,The National Suicide Prevention Lifeline is av...,qna_chitchat_Friendly,editorial:chitchat
4476,Aren't you going to sing to me?,"La la la, tra la la. I'm awesome at this.",qna_chitchat_Friendly,editorial:chitchat
7609,I'm feeling cross,Oh no! I'm sorry to hear that.,qna_chitchat_Friendly,editorial:chitchat


In [ ]:
#问题；回答；来源；闲聊

In [4]:
n_data = df_friend.shape[0]#9796
n_data

9796

In [5]:
df_friend #9796 rows × 4 columns 文件名

,Question,Answer,Source,Metadata
0,Lost the election because your speech was too ...,Swing and a miss.,qna_chitchat_Friendly,editorial:chitchat
1,This is so dull,Swing and a miss.,qna_chitchat_Friendly,editorial:chitchat
2,This is so uninteresting,Swing and a miss.,qna_chitchat_Friendly,editorial:chitchat
3,U basic,Swing and a miss.,qna_chitchat_Friendly,editorial:chitchat
4,Ugh so boring,Swing and a miss.,qna_chitchat_Friendly,editorial:chitchat
...,...,...,...,...
9791,I'm tired from work,I've heard really good things about naps.,qna_chitchat_Friendly,editorial:chitchat
9792,I'm totally drained,I've heard really good things about naps.,qna_chitchat_Friendly,editorial:chitchat
9793,I'm totally exhausted,I've heard really good things about naps.,qna_chitchat_Friendly,editorial:chitchat
9794,Zzzzz,I've heard really good things about naps.,qna_chitchat_Friendly,editorial:chitchat


In [10]:
df_friend.shape[0]  #  0--9796 1--4 二维

9796

In [14]:
df_friend['Question']  #纯问题

0       Lost the election because your speech was too ...
1                                         This is so dull
2                                This is so uninteresting
3                                                 U basic
4                                           Ugh so boring
                              ...                        
9791                                  I'm tired from work
9792                                  I'm totally drained
9793                                I'm totally exhausted
9794                                                Zzzzz
9795                                        I'm so sleepy
Name: Question, Length: 9796, dtype: object

In [16]:
df_friend['Question'].tolist()[:3] #变列表

['Lost the election because your speech was too boring',
 'This is so dull',
 'This is so uninteresting']

In [17]:
len(df_friend['Question'].tolist()) #9796

9796

In [18]:
df_friend['Answer'].tolist()[:3] #回答

['Swing and a miss.', 'Swing and a miss.', 'Swing and a miss.']

上为test

# 2. Preprocessing
This is just a very naive preprocessing.简单

In [13]:
question_list = df_friend['Question'].tolist()
answer_list = df_friend['Answer'].tolist()

In [27]:
import re
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# These are just common English contractions. There are many edge cases. i.e. University's working on it.
# 这些只是常见的英语缩略语。有很多边缘情况。即大学正在研究。

contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def pre_process(sent_list):
    output = []
    for sent in sent_list:
        sent = sent.lower()
        for word, new_word in contraction_dict.items():
            sent = sent.replace(word, new_word)
        sent = re.sub(r'[^\w\s]','',sent)        
        output.append(word_tokenize(sent))
    return output

input_token_list = pre_process(question_list)
answer_token_list = pre_process(answer_list)
output_token_list = [["<BOS>"] + s for s in answer_token_list]
target_token_list = [s + ["<EOS>"] for s in answer_token_list]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#test
input_token_list[:5]
# [['lost',
#   'the',
#   'election',
#   'because',
#   'your',
#   'speech',
#   'was',
#   'too',
#   'boring'],
#  ['this', 'is', 'so', 'dull'],
#  ['this', 'is', 'so', 'uninteresting'],
#  ['u', 'basic'],
#  ['ugh', 'so', 'boring']]

In [ ]:
answer_token_list[:5]
# [['swing', 'and', 'a', 'miss'],
#  ['swing', 'and', 'a', 'miss'],
#  ['swing', 'and', 'a', 'miss'],
#  ['swing', 'and', 'a', 'miss'],
#  ['swing', 'and', 'a', 'miss']]

In [28]:
output_token_list[:5]  #开头+BOS

[['<BOS>', 'swing', 'and', 'a', 'miss'],
 ['<BOS>', 'swing', 'and', 'a', 'miss'],
 ['<BOS>', 'swing', 'and', 'a', 'miss'],
 ['<BOS>', 'swing', 'and', 'a', 'miss'],
 ['<BOS>', 'swing', 'and', 'a', 'miss']]

In [29]:
target_token_list[:5] #结尾＋EOS

[['swing', 'and', 'a', 'miss', '<EOS>'],
 ['swing', 'and', 'a', 'miss', '<EOS>'],
 ['swing', 'and', 'a', 'miss', '<EOS>'],
 ['swing', 'and', 'a', 'miss', '<EOS>'],
 ['swing', 'and', 'a', 'miss', '<EOS>']]

In [30]:
MAX_LENGTH = max([len(s) for s in input_token_list] + [len(s) for s in target_token_list])

In [31]:
MAX_LENGTH#29

29

In [32]:
# set up a vocab to index dictionary 建立一个从词汇到索引的字典
word_to_ix = {"<BOS>": 0, "<EOS>":1}
for sentence in input_token_list+output_token_list:  #所有句子
    for word in sentence:     #没句子中的单词
        if word not in word_to_ix:    #没在里面就录入
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

In [ ]:
word_to_ix
# 'zoo': 877,
#  'store': 878,
#  'party': 879,
#  'concert': 880,
#  'beach': 881,
#  'bar': 882,
#  'church': 883,
#  'another': 884,
#  'tattoo': 885,
#  'car': 886,
#  'boat': 887,
#  'bring': 888,
#  'umbrella': 889

In [35]:
word_to_ix.keys()  #不要数字key

dict_keys(['<BOS>', '<EOS>', 'lost', 'the', 'election', 'because', 'your', 'speech', 'was', 'too', 'boring', 'this', 'is', 'so', 'dull', 'uninteresting', 'u', 'basic', 'ugh', 'ur', 'blah', 'gon', 'na', 'bore', 'me', 'to', 'death', 'wake', 'up', 'when', 'you', 'have', 'something', 'interesting', 'say', 'watching', 'paint', 'dry', 'more', 'entertaining', 'than', 'talking', 'are', 'trying', 'be', 'not', 'fun', 'fact', 'af', 'can', 'less', 'cool', 'story', 'bro', 'could', 'any', 'exciting', 'did', 'know', 'that', 'why', 'getting', 'tired', 'of', 'good', 'lord', 'holy', 'crap', 'i', 'am', 'amazed', 'enjoying', 'conversation', 'bored', 'with', 'falling', 'asleep', 'here', 'surprised', 'listening', 'like', 'a', 'ted', 'talk', 'from', 'color', 'beige', 'were', 'made', 'very', 'pretty', 'uneventful', 'quite', 'no', 'making', 'fall', 'such', 'drag', 'snooze', 'super', 'boringest', 'king', 'boredom', 'mayor', 'boringville', 'president', 'club', 'queen', 'really', 'yawn', 'as', 'sin', 'crazy', 'em

In [37]:
word_list[:6] #不要数字变列表

['<BOS>', '<EOS>', 'lost', 'the', 'election', 'because']

In [38]:
# construct token index lists for input, output and target# 为输入、输出和目标构建令牌索引列表
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

input_index = to_index(input_token_list, word_to_ix)    #input是问题
output_index = to_index(output_token_list, word_to_ix)  #开始+out是 回答 ['<BOS>', 'swing', 'and', 'a', 'miss']
target_index = to_index(target_token_list, word_to_ix)  # out+结束 ['swing', 'and', 'a', 'miss', '<EOS>']
# [59, 12, 45, 175, 63, 161, 2632, 1], 把词在词表中的索引找出来

In [39]:
input_token_list[0]

['lost',
 'the',
 'election',
 'because',
 'your',
 'speech',
 'was',
 'too',
 'boring']

In [40]:
output_token_list[0]

['<BOS>', 'swing', 'and', 'a', 'miss']

In [41]:
target_token_list[:5]

[['swing', 'and', 'a', 'miss', '<EOS>'],
 ['swing', 'and', 'a', 'miss', '<EOS>'],
 ['swing', 'and', 'a', 'miss', '<EOS>'],
 ['swing', 'and', 'a', 'miss', '<EOS>'],
 ['swing', 'and', 'a', 'miss', '<EOS>']]

In [ ]:
#to_index(target_token_list, word_to_ix)

# 3. Model

In [43]:
import torch
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 3.1 Encoder

In [ ]:
#训练过程，把它提上来了。
# hidden_size = 50
# embedding = nn.Embedding(len(word_to_ix), hidden_size)
# encoder1 = EncoderRNN(len(word_to_ix), hidden_size, embedding).to(device)
# attn_decoder1 = AttnDecoderRNN(hidden_size, len(word_to_ix),embedding, dropout_p=0.1).to(device)

# trainIters(encoder1, attn_decoder1, 10000, print_every=500)

In [49]:
#拆解
hidden_size = 50
embedding = nn.Embedding(len(word_to_ix), hidden_size)  #2668，50
gru = nn.GRU(hidden_size, hidden_size)  #GRU(50, 50)
gru


GRU(50, 50)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, embedding):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden) 
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)    

## 3.2 Decoder

In [ ]:

class AttnDecoderRNN(nn.Module):
    ATTN_TYPE_DOT_PRODUCT = "Dot Product"
    ATTN_TYPE_SCALE_DOT_PRODUCT = "Scale Dot Product"

    def __init__(self, hidden_size, output_size, embedding, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = embedding
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)


    def cal_attention(self, hidden, encoder_hiddens, method):
        if method == AttnDecoderRNN.ATTN_TYPE_DOT_PRODUCT:
            # bmm: https://pytorch.org/docs/master/generated/torch.bmm.html
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        return concat_output

    def forward(self, input, hidden, encoder_hiddens):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        _, hidden = self.gru(embedded, hidden)

        concat_output = self.cal_attention(hidden, encoder_hiddens, AttnDecoderRNN.ATTN_TYPE_DOT_PRODUCT)

        output = F.log_softmax(self.out(concat_output), dim=1)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

上下一样

In [1]:
class AttnDecoderRNN(nn.Module):
    ATTN_TYPE_DOT_PRODUCT = "Dot Product"
    ATTN_TYPE_SCALE_DOT_PRODUCT = "Scale Dot Product"

    def __init__(self, hidden_size, output_size, embedding, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = embedding
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)


    def cal_attention(self, hidden, encoder_hiddens, method):
        if method == AttnDecoderRNN.ATTN_TYPE_DOT_PRODUCT:
            # bmm: https://pytorch.org/docs/master/generated/torch.bmm.html
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        elif method == AttnDecoderRNN.ATTN_TYPE_SCALE_DOT_PRODUCT:
            # COMPLETE THIS PART - Scale Dot Product calculation method # 完成这部分 - 比例点积的计算方法
            attn_weights =
            attn_output = 
            concat_output =

        return concat_output

    def forward(self, input, hidden, encoder_hiddens):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        _, hidden = self.gru(embedded, hidden)

        ## The following attention score calculation method is Dot Product for now
        ## Please change it into Scale Dot Product calculation method
        ##下面的注意分数计算方法暂时是点积法
        ##请将其改为尺度点积计算方法Scale Dot Product
        concat_output = self.cal_attention(hidden, encoder_hiddens, AttnDecoderRNN.ATTN_TYPE_DOT_PRODUCT)
        # concat_output = self.cal_attention(hidden, encoder_hiddens, AttnDecoderRNN.ATTN_TYPE_SCALE_DOT_PRODUCT)

        output = F.log_softmax(self.out(concat_output), dim=1)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)



SyntaxError: ignored

后面是老师代码


## 3.3 Train Function

In [ ]:

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_hiddens = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for i in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
        encoder_hiddens[i] = encoder_hidden[0, 0]

    decoder_input = torch.tensor([[0]], device=device)

    decoder_hidden = encoder_hidden

    # Teacher forcing: Feed the target as the next input
    for i in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_hiddens)
        loss += criterion(decoder_output, target_tensor[i])
        decoder_input = target_tensor[i]  # Teacher forcing

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

## 3.4 Train Iterations Function

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:

import random
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        random_choice_ix = random.choice(range(n_data))
        input_index_r = [[ind] for ind in input_index[random_choice_ix]]
        target_index_r = [[ind] for ind in target_index[random_choice_ix]]
        
        input_tensor = torch.LongTensor(input_index_r).to(device)
        target_tensor = torch.LongTensor(target_index_r).to(device)

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


# 4. Training Process

In [ ]:
hidden_size = 50
embedding = nn.Embedding(len(word_to_ix), hidden_size)
encoder1 = EncoderRNN(len(word_to_ix), hidden_size, embedding).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, len(word_to_ix),embedding, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 10000, print_every=500)

0m 9s (- 3m 9s) (500 5%) 5.5442
0m 19s (- 2m 58s) (1000 10%) 3.6570
0m 29s (- 2m 49s) (1500 15%) 2.7173
0m 39s (- 2m 37s) (2000 20%) 2.0946
0m 48s (- 2m 26s) (2500 25%) 1.9571
0m 58s (- 2m 16s) (3000 30%) 1.5846
1m 8s (- 2m 6s) (3500 35%) 1.3695
1m 17s (- 1m 56s) (4000 40%) 1.2728
1m 27s (- 1m 47s) (4500 45%) 1.1528
1m 37s (- 1m 37s) (5000 50%) 1.0607
1m 46s (- 1m 27s) (5500 55%) 0.9557
1m 56s (- 1m 17s) (6000 60%) 0.9602
2m 7s (- 1m 8s) (6500 65%) 0.7885
2m 17s (- 0m 58s) (7000 70%) 0.7891
2m 27s (- 0m 49s) (7500 75%) 0.7809
2m 36s (- 0m 39s) (8000 80%) 0.7507
2m 46s (- 0m 29s) (8500 85%) 0.7150
2m 57s (- 0m 19s) (9000 90%) 0.6768
3m 7s (- 0m 9s) (9500 95%) 0.6630
3m 17s (- 0m 0s) (10000 100%) 0.6352


# 5. Evaluation

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_sent = pre_process([sentence])[0]
        intput_index = [word_to_ix[word] for word in input_sent]
        input_tensor = torch.LongTensor([[ind] for ind in intput_index]).to(device)

        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_hiddens = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_hiddens[ei] += encoder_hidden[0, 0]

        decoder_input = torch.tensor([[0]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_hiddens)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == 1:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(word_list[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
sentence1 = "Are you in love with me?"
sentence2 = "Who do you love"
sentence3 = "Are you able to check my email?"	
sentence4 = "You're the best"

print(evaluate(encoder1, attn_decoder1, sentence1, max_length=MAX_LENGTH))
print(evaluate(encoder1, attn_decoder1, sentence2, max_length=MAX_LENGTH))
print(evaluate(encoder1, attn_decoder1, sentence3, max_length=MAX_LENGTH))
print(evaluate(encoder1, attn_decoder1, sentence4, max_length=MAX_LENGTH))

['i', 'am', 'a', 'bot', 'who', 'was', 'created', 'by', 'humans', '<EOS>']
['if', 'it', 'were', 'a', 'contest', 'which', 'it', 'is', 'not', 'you', 'would', 'still', 'probably', 'win', '<EOS>']
['i', 'am', 'a', 'bot', 'who', 'was', 'created', 'by', 'humans', '<EOS>']
['if', 'it', 'were', 'a', 'contest', 'which', 'it', 'is', 'not', 'you', 'would', 'still', 'probably', 'win', '<EOS>']


In [ ]:
attn_weights = F.softmax((torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)) / math.sqrt(encoder_hiddens.shape[1])), dim=-1)
attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0)) 
concat_output = torch.cat((attn_output[0], hidden[0]), 1)

#就是n变了除了个什么n

In [ ]:
attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0))/np.sqrt(hidden_size),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0)) 
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        return concat_output
